In [4]:
import tensorflow as tf

# CIFAR-10 데이터를 다운로드하고 데이터를 불러옵니다.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data() # np.array

# 이미지들을 float32 데이터 타입으로 변경합니다.
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')

# [0,255] 사이의 값을 [0,1] 사이의 값으로 Normalize 합니다.(세심한 소수점 잊지말기)
x_train, x_test = x_train/255., x_test/255.

# 스칼라 형태의 레이블(0~9)를 one-hot Encoding 형태로 변환합니다.
y_train_one_hot = tf.squeeze(tf.one_hot(y_train,10), axis=1) # tf.squeeze는 개수가 1인 dimension을 제거해서 차원 줄여줌
y_test_one_hot = tf.squeeze(tf.one_hot(y_test,10),axis=1)

# y_train_one_hot >> <tf.Tensor 'Squeeze:0' shape=(50000, 10) dtype=float32>
# x_train.shape >> (50000, 32, 32, 3)
# tf.data API를 이용해서 데이터를 섞고 batch형태로 가져옵니다.
train_data = tf.data.Dataset.from_tensor_slices((x_train,y_train_one_hot))
train_data = train_data.repeat().shuffle(50000).batch(128)
# repeat을 사용하면 dataset이 몇 번 반복해서 사용될 지 정할 수 있다. 파라미터가 없다면 계속 반복하고 보통 계속 반복시키고 epoch값을 직접 제어하는 것이 좋다.
# shuffle을 사용하면 설정된 epoch마다 dataset을 섞을 수 있다. Dataset의 shuffle은 overfitting을 피할 때 매우 중요
train_data_iter = iter(train_data)

test_data = tf.data.Dataset.from_tensor_slices((x_test,y_test_one_hot))
test_data = test_data.batch(1000) # 아까는 train만 배치를 했었는데 이번엔 테스트도
test_data_iter = iter(test_data)


In [6]:
# 중간에 레이어를 바꿀 수 있는지 궁금
# tf.keras.Model을 이용해서 CNN모델을 정의합니다

class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        # 천천히 봐봅시다.
        # 첫번째 convolutional layer - 하나의 RGB이미지를 64개의 특징들(feature)로 맵핑(mapping)합니다. padding은 차원을 축소할 것이 아니라면 'same'
        self.conv_layer_1 = tf.keras.layers.Conv2D(filters = 64, kernel_size=5, strides=1, padding='same',activation='relu') #2D convolution layer (e.g. spatial convolution over images).
        # This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs.
        self.pool_layer_1 = tf.keras.layers.MaxPool2D(pool_size = (3,3),strides=2)
        # max pooling, is a pooling operation that calculates the maximum, or largest, value in each patch of each feature map. The results are down sampled or pooled feature maps that highlight the most present feature in the patch, not the average presence of the feature in the case of average pooling.
        
        # 두번째 convolutional layer - 64개의 특징들(feature)을 64개의 특징들로 맵핑 -> 왜 64개이지?
        self.conv_layer_2 = tf.keras.layers.Conv2D(filters = 64, kernel_size =5, strides = 1, padding='same',activation='relu')
        self.pool_layer_2 = tf.keras.layers.MaxPool2D(pool_size = (3,3),strides=2)
        # Convolution을 거쳐서 나온 activation maps이 있을 때,이를 이루는 convolution layer을 resizing하여 새로운 layer를 얻는 것 : pooling(sampling)
        # 필터랑 커널이 같은거래!
        # 아 stride라는 건 그림 전체를 하나의 큰 필터로만 보는 게 아니라, 예를 들어 300*300 사진을 3*3 으로 본다면, 3*3 짜리를 한 칸씩 내려서 볼 건지, 두 칸씩 내릴 건지 필터를 적용한 후의 결과를 feature map 혹은 activation map이라 한다
        
        # 세번째 convolutional layer
        self.conv_layer_3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same',activation='relu')
        # 네번째 convolutional layer 
        self.conv_layer_4 = tf.keras.layers.Conv2D(filters=128, kernel_size = 3, strides=1, padding='same',activation='relu')
        # 다섯번째 convolutional layer 
        self.conv_layer_5 = tf.keras.layers.Conv2D(filters=128, kernel_size = 3, strides=1, padding='same',activation='relu')
        
        # Fully Connected Layer 1: 2 번의 downsampling(샘플링 주기를 낮추는 것) 이후에, 우리의 32*32 이미지는 8*8*128 특징맵(feature map)이 됩니다.
        # 이를 384 개 특징들로 맵핑합니다.
        self.flatten_layer = tf.keras.layers.Flatten()
        self.fc_layer_1 = tf.keras.layers.Dense(384, activation='relu')
        self.dropout = tf.keras.layers.Dropout(0.2) # 뉴럴 네트워크가 학습중일때, 랜덤하게 뉴런을 꺼서 학습을 방해함으로써, 학습이 학습용 데이타에 치우치는 현상을 막아준다.
        
        # Fully Connected Layer 2 : 384 개의 특징들(feature)을 10개의 클래스 - airplane, automobile, bird, ,, 로 맵핑합니다
        self.output_layer = tf.keras.layers.Dense(10, activation=None)
        
    def call(self, x, is_training): # 파라미터 넣으면서 아웃풋 반환
        #입력 이미지
        h_conv1 = self.conv_layer_1(x)
        h_pool1 = self.pool_layer_1(h_conv1)
        h_conv2 = self.conv_layer_2(h_pool1)
        h_pool2 = self.pool_layer-2(h_conv2)
        h_conv3 = self.conv_layer_3(h_pool2)
        h_conv4 = self.conv_layer_4(h_conv3)
        h_conv5 = self.conv_layer_5(h_conv4)
        h_conv5_flat = self.flatten_layer(h_conv5)
        h_fc1 = self.fc_layer_1(h_conv5_flat)
        # Dropout : 모델의 복잡도를 컨트롤, 특징들의 co-adaptation(동조, 비슷한 특징에 집중해버리는)을 방지
        h_fc1_drop = self.dropout(h_fc1, training=is_training)
        logits = self.output_layer(h_fc1_drop)
        y_pred = tf.nn.softmax(logits)
        
        # 아래에 logit이 필요하므로 logits도 return 
        # 보통은 트레인에서 드랍아웃을 한다

        return y_pred, logits



In [7]:
# cross entropy 손실함수를 정의한다.
@tf.function
def cross_entropy_loss(logits, y):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# 최적화를 위한 RMSprop 옵티마이저 정의
optimizer = tf.optimizers.RMSprop(1e-3)

# 최적화를 위한 function을 정의한다.
@tf.function
def train_step(model,x,y,is_training):
    with tf.GradientTape() as tape:
        y_pred, logits = model(x, is_training)
        loss = cross_entropy_loss(logits,y)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients,model.trainable_variables))
        # loss에 대하여 학습가능한 weight 의 gradient를 알고싶다면 GradientTape scope를 정의해야 합니다. 
        # optimizer 객체를 사용하여 model.trainable_variables 를 통해 업데이트되는 gradient 를 사용할 수 있습니다.
        
# 모델의 정확도를 출력하는 함수 정의
@tf.function
def compute_accuracy(y_pred,y):
    correct_prediction = tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return accuracy

AttributeError: module 'tensorflow' has no attribute 'optimizers'

In [ ]:
CNN_model = CNN()

# 10000step 만큼 최적화 수행
for i in range(10000):
    batch_x, batch_y = next(train_data_iter)
    
    # 100step마다 training 데이터 셋에 대한 정확도와 loss를 출력합니다.
    if i % 100 == 0:
        train_accuracy = compute_accuracy(CNN_model(batch_x,False)[0],batch_y)
        loss_print = cross_entropy_loss(CNN_model(batch_x, False)[1], batch_y)
        
        print('반복(Epoch): %d, 트레이닝 데이터 정확도: %f, 손실 함수(loss): %f'%(i,train_accuracy, loss_print))
        
    # 20% 확률의 dropout 이용해 학습 진행
    train_step(CNN_model, batch_x, batch_y, True)
    
# 학습이 끝나면 테스트 데이터(10000개)에 대한 정확도 출력
test_accuracy = 0.0
for i in range(10):
    test_batch_x, test_batch_y = next(test_data_iter)
    test_accuracy = test_accuracy + compute_accuracy(CNN_model(test_batch_x, False)[0], test_batch_y).numpy()
test_accuracy = test_accuracy / 10
print('테스트 데이터 정확도: %f'%test_accuracy)

In [ ]:
#참고
#출처: https://bcho.tistory.com/1149 [조대협의 블로그]